In [50]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import torch
import torchvision.transforms as transforms
import numpy as np
import torchvision

In [51]:
mean = [0.4914, 0.4822, 0.4465]
std = [0.2023, 0.1994, 0.2010]

In [52]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10), 
    transforms.RandomCrop(32, padding=4),  
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  
    transforms.ToTensor(),  
    transforms.Normalize(mean=mean, std=std)
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

In [53]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=val_transform)

num_samples_per_class = 25
num_classes = 2
random_classes = np.random.choice(range(10), num_classes, replace=False)

Files already downloaded and verified
Files already downloaded and verified


In [54]:
selected_indices_train = []
for i in random_classes:
    class_indices = np.where(np.array(trainset.targets) == i)[0]
    selected_indices_train.extend(np.random.choice(class_indices, num_samples_per_class, replace=False))

selected_trainset = torch.utils.data.Subset(trainset, selected_indices_train)
trainloader = torch.utils.data.DataLoader(selected_trainset, batch_size=len(selected_trainset), shuffle=False)
train_features, train_labels = next(iter(trainloader))
train_features = train_features.numpy().reshape(len(selected_trainset), -1)  

selected_indices_test = []
for i in random_classes:
    class_indices = np.where(np.array(testset.targets) == i)[0]
    selected_indices_test.extend(class_indices)

selected_testset = torch.utils.data.Subset(testset, selected_indices_test)

In [55]:
print(train_features.shape)

(50, 3072)


In [56]:
testloader = torch.utils.data.DataLoader(selected_testset, batch_size=len(selected_testset), shuffle=False)
test_features, test_labels = next(iter(testloader))
test_features = test_features.numpy().reshape(len(selected_testset), -1) 

k = 5
knn_classifier = KNeighborsClassifier(n_neighbors=k)


knn_classifier.fit(train_features, train_labels)
predictions = knn_classifier.predict(test_features)
accuracy = accuracy_score(test_labels, predictions)
print("Accuracy: {} %".format(accuracy*100))


Accuracy: 52.55 %


In [57]:
print(test_features.shape)

(2000, 3072)


In [ ]:
## Working in TestBed

In [ ]:
def train(model, device, train_loader, optimizer, epoch, display=True):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
    if display:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
          epoch, batch_idx * len(data), len(train_loader.dataset),
          100. * batch_idx / len(train_loader), loss.item()))

In [ ]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, size_average=False).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return 100. * correct / len(test_loader.dataset)